In [56]:
# !pip install investpy

import urllib.request, json , time, os, difflib, itertools
import pandas as pd
from multiprocessing.dummy import Pool
from datetime import datetime
from datetime import date
import dateutil
from datetime import timezone
import time
import pathlib
try:
    import httplib
except:
    import http.client as httplib

In [4]:
def check_internet():
    conn = httplib.HTTPConnection("www.google.com", timeout=5)
    try:
        conn.request("HEAD", "/")
        conn.close()
        print("connected")
        return True
    except:
        conn.close()
        print("not connected")
        return False

Get data from Yahoo finance

In [57]:
def get_historic_price(query_url,json_path,csv_path):
    if not os.path.exists(json_path):
        os.makedirs(json_path)
    if not os.path.exists(csv_path):
        os.makedirs(csv_path)
        
    stock_id=query_url.split("&period")[0].split("symbol=")[1]

    if os.path.exists(csv_path+stock_id+'.csv') and os.stat(csv_path+stock_id+'.csv').st_size != 0:
        print("<<<  Historical data of "+stock_id+" already exists")
        return
    
    while not check_internet():
        print("Could not connect, trying again in 5 seconds...")
        time.sleep(5)

    try:
        with urllib.request.urlopen(query_url) as url:
            parsed = json.loads(url.read().decode())
    
    except:
        print("|||  Historical data of "+stock_id+" doesn't exist")
        return
    
    else:
        if os.path.exists(json_path+stock_id+'.json') and os.stat(json_path+stock_id+'.json').st_size != 0:
            os.remove(json_path+stock_id+'.json')
        
        with open(json_path+stock_id+'.json', 'w') as outfile:
            json.dump(parsed, outfile, indent=4)
        
        try:
            Date=[]
            for i in parsed['chart']['result'][0]['timestamp']:
                Date.append(datetime.utcfromtimestamp(int(i)).strftime('%d-%m-%Y'))

            Low=parsed['chart']['result'][0]['indicators']['quote'][0]['low']
            Open=parsed['chart']['result'][0]['indicators']['quote'][0]['open']
            Volume=parsed['chart']['result'][0]['indicators']['quote'][0]['volume']
            High=parsed['chart']['result'][0]['indicators']['quote'][0]['high']
            Close=parsed['chart']['result'][0]['indicators']['quote'][0]['close']
            Adjusted_Close=parsed['chart']['result'][0]['indicators']['adjclose'][0]['adjclose']

            df=pd.DataFrame(list(zip(Date,Low,Open,Volume,High,Close,Adjusted_Close)),columns =['Date','Low','Open','Volume','High','Close','Adjusted Close'])

            if os.path.exists(csv_path+stock_id+'.csv'):
                os.remove(csv_path+stock_id+'.csv')
            df.to_csv(csv_path+stock_id+'.csv', sep=',', index=None)
            print(">>>  Historical data of "+stock_id+" saved")
        
        except:
            print(">>>  Historical data of "+stock_id+" could not be saved")
        
        return

# print("retrieving trading data from " + date.today().strftime("%d/%m/%Y") +" to " + (date.today() - delta).strftime("%d/%m/%Y") )
# print(query_url)
# get_historic_price(query_url,str(pathlib.Path().absolute())+"/JSON/",str(pathlib.Path().absolute())+"/CSV/")

retrieving trading data from 02/01/2021 to 02/07/2020
https://query2.finance.yahoo.com/v8/finance/chart/SAMPN0000.CM?symbol=SAMPN0000.CM&period1=1593673200&period2=1609574400&interval=1d
<<<  Historical data of SAMPN0000.CM already exists


Pull data from http://investing.com

In [182]:
import investpy
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.offline import plot
import plotly
import math
from datetime import datetime, timedelta
from dateutil import relativedelta

def draw_line_plot(data,title,y_axis,row,col,fig):
    fig.append_trace(go.Scatter(
        name =title,
        x=data['Date'],
        y=data[y_axis],
    ), row=row, col=col)
    
    return fig


symbols_and_names = {'SAMP':"Sampath",'JKH':"Jhon Keels",'DIPD':"DIPD",'SOCL':"Softlogic Capitals"
                     ,'BROW':"BIL",'HNB':"HNB",'ABAN':"Abans Electricals PLC",'AFSL':"Abans Finance PLC"
                     ,'ABLT':"Amana Bank Ltd","CHL":"Ceylon Hospitals","CPRT":"Ceylon Printers",
                    'HUNA':}

symbols = list(symbols_and_names.keys())
symbols_names = list(symbols_and_names.values())
to_ = date.today().strftime("%d/%m/%Y")
delta = relativedelta.relativedelta(months=6)
from_ = (date.today() - delta).strftime("%d/%m/%Y")
sym_data_frame = {}
for sym in symbols:
    df = investpy.get_stock_historical_data(stock=sym,
                                        country='Sri Lanka',
                                        from_date=from_,
                                        to_date=to_)

    df = df.reset_index()
    sym_data_frame[sym]=df
   
rows = 5
cols = 3
fig = make_subplots(rows=rows, cols=cols,subplot_titles=symbols_names)
index = 0
for k,v in sym_data_frame.items():
    row = math.floor(index/cols)+1
    col = index%cols+1
    draw_line_plot(v,k,"Close",row,col,fig)
    index +=1
 
    
fig.update_layout(
        showlegend=False,
        xaxis=dict(
#         title=title,
    #     tickmode='linear',
    #         tick0 = 0.5,
            dtick = 'M1',
         
        showticklabels= True
        ),
        height=900, 
        width=1000,)   
fig.update_xaxes(dtick = 'M1',tickangle = 25)
fig = go.Figure(fig)
fig.show()